In [18]:
import gym
from gym import spaces
import numpy as np
import pygame
import random

In [19]:
class FlappyBirdEnv(gym.Env):
    def __init__(self):
        super(FlappyBirdEnv, self).__init__()

        pygame.init()
        self.screen_width = 288
        self.screen_height = 512
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption("Flappy Bird - Custom Env")

        self.gravity = 1
        self.jump_power = -10
        self.bird_y = self.screen_height // 2
        self.bird_vel = 0

        self.pipe_gap = 150
        self.pipe_width = 50
        self.pipe_x = self.screen_width
        self.pipe_height = random.randint(100, 300)

        low = np.array([0, -15, 0, 100], dtype=np.float32)
        high = np.array([self.screen_height, 15, self.screen_width, 300], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        self.action_space = spaces.Discrete(2)

        self.clock = pygame.time.Clock()

    def reset(self):
        self.bird_y = self.screen_height // 2
        self.bird_vel = 0
        self.pipe_x = self.screen_width
        self.pipe_height = random.randint(100, 300)
        return self._get_obs()

    def _get_obs(self):
        return np.array([
            self.bird_y,
            self.bird_vel,
            self.pipe_x,
            self.pipe_height
        ], dtype=np.float32)

    def step(self, action):
        reward = 1

        if action == 1:
            self.bird_vel = self.jump_power

        self.bird_vel += self.gravity
        self.bird_y += self.bird_vel
        self.pipe_x -= 5

        if self.pipe_x < -self.pipe_width:
            self.pipe_x = self.screen_width
            self.pipe_height = random.randint(100, 300)

        done = False
        if self.bird_y > self.screen_height or self.bird_y < 0:
            done = True
            reward = -100

        if (self.pipe_x < 50 < self.pipe_x + self.pipe_width):
            if not (self.pipe_height < self.bird_y < self.pipe_height + self.pipe_gap):
                done = True
                reward = -100

        return self._get_obs(), reward, done, {}

    def render(self, mode='human'):
        self.screen.fill((0, 0, 0))
        pygame.draw.circle(self.screen, (255, 255, 0), (50, int(self.bird_y)), 10)
        pygame.draw.rect(self.screen, (0, 255, 0), (self.pipe_x, 0, self.pipe_width, self.pipe_height))
        pygame.draw.rect(self.screen, (0, 255, 0), (self.pipe_x, self.pipe_height + self.pipe_gap,self.pipe_width, self.screen_height))
        pygame.display.flip()
        self.clock.tick(30)

    def close(self):
        pygame.quit()


In [20]:
env = FlappyBirdEnv()
obs = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)
    env.render()

env.close()

In [21]:
!jupyter nbconvert --to script flappy_custom_env.ipynb

[NbConvertApp] Converting notebook flappy_custom_env.ipynb to script
[NbConvertApp] Writing 3040 bytes to flappy_custom_env.py
